In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Hyperparameters
EPOCHS = 20 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

In [3]:
def tokenizer(stop_words):
    t = get_tokenizer("spacy", language="en_core_web_sm")
    def f(text):
        import re
        text = re.sub(r'[^\w\s]', '', text)
        text = text.lower().split()
        text = ' '.join(
            [w for w in text if w not in stop_words and len(w) >= 3])
        return t(text)
    return f


def yield_tokens(data_iter, tokenizer):
    for _, text in data_iter:
        yield tokenizer(text)

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

def train(dataloader, criterion, optimizer, epoch):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 10
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader, criterion):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count


In [26]:
class IMDBDataset(Dataset):
    def __init__(self, df: pd.DataFrame, text_field, label_field):
        self.df = df
        self.text_field = text_field
        self.label_field = label_field

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx][self.text_field]
        label = self.df.iloc[idx][self.label_field]
        return label, text

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_size, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.rnn = nn.RNN(embed_dim, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        output, hidden = self.rnn(embedded)
        return self.fc(output), hidden

In [5]:
imdb_df = pd.read_csv("IMDB_Dataset.csv", encoding='latin-1')
imdb_df['sentiment'] = imdb_df['sentiment'].map({'positive': 1, 'negative': 0})
stop_words = open("stopwords.txt", "r").readlines()
all_categories = ['neg', 'pos']
tokenizer_fn = get_tokenizer(tokenizer(stop_words))

In [6]:
train_split, test_split = train_test_split(imdb_df, test_size=0.2, random_state=1)
train_ds = IMDBDataset(train_split, 'review', 'sentiment')
test_ds = IMDBDataset(test_split, 'review', 'sentiment')

In [7]:
vocab = build_vocab_from_iterator(yield_tokens(train_ds, tokenizer_fn), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [8]:
text_pipeline = lambda x: vocab(tokenizer_fn(x))
label_pipeline = lambda x: int(x)

In [9]:
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

In [27]:
num_class = len(set([label for (label, text) in train_ds]))
vocab_size = len(vocab)
emsize = 128
nhid = 64
model = TextClassificationModel(vocab_size, emsize, nhid, num_class).to(device)

In [29]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_dataset = to_map_style_dataset(train_ds)
test_dataset = to_map_style_dataset(train_ds)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train],
                 generator=torch.Generator().manual_seed(1))

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=False, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader, criterion, optimizer, epoch)
    accu_val = evaluate(valid_dataloader, criterion)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)


| epoch   1 |    10/   30 batches | accuracy    0.705
| epoch   1 |    20/   30 batches | accuracy    0.727
-----------------------------------------------------------
| end of epoch   1 | time: 76.76s | valid accuracy    0.700 
-----------------------------------------------------------
| epoch   2 |    10/   30 batches | accuracy    0.722
| epoch   2 |    20/   30 batches | accuracy    0.706
-----------------------------------------------------------
| end of epoch   2 | time:  4.75s | valid accuracy    0.535 
-----------------------------------------------------------
| epoch   3 |    10/   30 batches | accuracy    0.714
| epoch   3 |    20/   30 batches | accuracy    0.791
-----------------------------------------------------------
| end of epoch   3 | time:  4.30s | valid accuracy    0.745 
-----------------------------------------------------------
| epoch   4 |    10/   30 batches | accuracy    0.814
| epoch   4 |    20/   30 batches | accuracy    0.801
-------------------------

: 

In [12]:
print('Checking the results of train dataset.')
accu_train = evaluate(train_dataloader, criterion)
print(f'train accuracy {accu_train}')

Checking the results of train dataset.
train accuracy 0.7931578947368421


In [13]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader, criterion)
print(f'test accuracy {accu_test}')

Checking the results of test dataset.
test accuracy 0.79275
